In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
import pandas as pd
import numpy as np
from numpy import zeros
from numpy import ones
from numpy import hstack
from numpy.random import rand
from numpy.random import randn
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [ ]:
filename="DB_modified.csv"
dataset = pd.read_csv(filename, header=None)
dataset =np.array(dataset)[1:-1]

dataset=np.asfarray(dataset,float)
dataset.shape

In [ ]:
X=dataset[:,:7]
# print("BeforeSTD:", X)
# X=(X-np.mean(X))/np.std(X)
# X=dataset[:,2]
X[:,1]=(X[:,1]-np.mean(X[:,1]))/np.std(X[:,1])
X[:,2]=(X[:,2]-np.mean(X[:,2]))/np.std(X[:,2])
X[:,3]=(X[:,3]-np.mean(X[:,3]))/np.std(X[:,3])
# X[:,1]=X[:,1]/np.max(X[:,1])
# y=dataset[:,6]
# Vthin=dataset[:,-2]
# np.concatenate((X, np.expand_dims(Vthin, 1)), 1)
# print(X)
# print(y.shape)
# X=np.expand_dims(X, 1)
# print("AfterSTD:",X)

In [ ]:
def define_discriminator(n_inputs=7):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def define_generator(latent_dim, n_outputs=7):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

In [ ]:
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [ ]:
def generate_real_samples(n):
    X_return = X[np.random.randint(0, X.shape[0], size=n)]
    y = ones((n, 1))
    return X_return, y


In [ ]:
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    return x_input

In [ ]:
def generate_fake_samples(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n, 1))
    return X, y

In [ ]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generate_real_samples(n)
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    # scatter plot real and fake data points
    pyplot.scatter(x_real[:, 3], x_real[:, 6], color='red')
    pyplot.scatter(x_fake[:, 3], x_fake[:,6], color='blue')
    pyplot.show()

In [ ]:
def train(g_model, d_model, gan_model, latent_dim, n_epochs=4000, n_batch=64, n_eval=50):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_real_samples(half_batch)
       
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
#         print(x_gan.shape, y_gan.shape)
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)

In [ ]:
latent_dim = 10
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)